# 01 - First notebook

In [1]:
import numpy as np
import fiona
import rasterio
import rasterio.features
from shapely.geometry import shape, mapping, Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [2]:
import os

In [3]:
!pwd

/home/drugo/radar-precipitation


In [4]:
path = '/home/drugo/radar-precipitation'

In [5]:
os.chdir(path)

In [6]:
def read_tiff_file(file_path):
    with open(file_path, 'r') as f:
        print(f.read())

In [7]:
def read_tiff_file(file_path):
    centroids = []
    with rasterio.open(file_path) as src:
        crs = src.crs
        src_band = src.read(1)
        src_band[src_band<=1]=255
        src_band[src_band<255]=1
        unique_values = np.unique(src_band)
        unique_values = unique_values[[0]]
        shapes = list(rasterio.features.shapes(src_band, transform=src.transform))
        
        shp_polygon_schema = {
            'geometry': 'MultiPolygon',
            'properties': {'id':'int'}
        }
        
        with fiona.open('../polygons/'+file_path.replace('.tiff','.shp'),'w','ESRI Shapefile',shp_polygon_schema) as shp:
            for pixel_value in unique_values:
                polygons = [shape(geom) for geom, value in shapes
                            if value == pixel_value]
                for i in range(len(polygons)):
                    multipolygon = MultiPolygon(polygons)    
                                
            shp.write({
                'geometry': mapping(multipolygon),
                'properties': {'id': int(file_path[:-5])}
            })
            
        shp_centroid_schema = {
            'geometry': 'Point',
            'properties': {'id':'int'}
        }
        centroids.append(multipolygon.centroid)
        
        with fiona.open('../centroids/'+file_path.replace('.tiff','_centroid.shp'),'w','ESRI Shapefile',shp_centroid_schema) as shp:
            shp.write({
                'geometry': mapping(multipolygon.centroid),
                'properties': {'id': int(file_path[:-5])}
            })

In [8]:
for file in os.listdir():
    if file.endswith(".tiff"):
        print(file)
        read_tiff_file(file)

############################################################

############################################################

############################################################